In [1]:
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import re
import scipy.constants as c
from scipy.io import loadmat
from scipy.optimize import curve_fit
from scipy.signal import welch
from scipy.signal import find_peaks
from scipy.signal import periodogram
from scipy.interpolate import interp1d
import scipy.signal as sig
import math
import string

# Gui stuff.
import tkinter
from PIL import ImageTk, Image
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
#plt.style.use("dark_background")


# Split into like 1000 sections for each trace, average those and find variance of each slice.
# Allan variance (do it).

In [2]:
# Functions to be fitted.
def sqq(omega,omegaq,Gammacm,Tcm,M):
    return ((abs(Gammacm)*c.k*abs(Tcm))/(np.pi*abs(M)))/((omega**2-abs(omegaq)**2)**2+(abs(Gammacm)**2)*(abs(omega)**2))

def ssqq(omega,omegaq,Gammacm,a):
    return ((abs(Gammacm)*abs(a))/((omega**2-abs(omegaq)**2)**2+(abs(Gammacm)**2)*(abs(omega)**2)))

In [8]:
# Misc.
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))

def progress_bar(i,L,w):
    pr = i/L
    print(2*w*' ', end = '\r')
    print('[{}{}]{}/100%'.format(int(w*pr)*'#',int(w*(1-pr))*'.',round(100*pr,1)),end='\r')

def count_e(nested_list):
    return sum(count_elements_nested_list(item) if isinstance(item, list) else 1 for item in nested_list)

class Rod_Gamas:
    def __init__(self, T_s, l, R, D, p):
        self.l = l
        self.R = R
        self.p = p
        self.T = 300
        self.M = np.pi * R**2 * l * D
        self.alpha_c = 0.65 # Assuming surrounding gas is N_2.
        self.gamma_s = np.sqrt(T_s/self.T) # T_s = surface temp, # T = gas temp; (maybe ~1).
        self.m = 28.0134/(1000*c.N_A) # Mass of gas molecule (N_2).
        self.ng  = p/(c.k*self.T) # Gas density.

    def cm_t(self):
        return (self.ng*self.R*self.l*np.sqrt(2*np.pi*self.m*c.k*self.T)/self.M) * (2+self.alpha_c(-1/2 + np.pi*self.gamma_s/4 + self.R/self.l))

    def cm_p(self):
        return (self.ng*self.R*self.l*np.sqrt(2*np.pi*self.m*c.k*self.T)/self.M) * (4*self.R/self.l+self.alpha_c*(1 - 2*self.R/self.l + np.pi*self.gamma_s*self.R/(2*self.l)))

    def rot_t(self):
        return (self.ng*self.R*self.l*np.sqrt(2*np.pi*self.m*c.k*self.T)/self.M) * (self.l**2/(3*self.R**2 + self.l**2))* (2 + 12*(self.R/self.l)**3 + self.alpha_c*(-1/2 + np.pi*self.gamma_s/4 + 3*self.R/self.l + 6*(self.R/self.l)**2 + (3*np.pi*self.gamma_s/2 - 6)*(self.R/self.l)**3))

    def rot_p(self):
        return (self.ng*self.R*self.l*np.sqrt(2*np.pi*self.m*c.k*self.T)/self.M) * self.alpha_c*(2+ self.R/self.l)
 
class data_loader:
    def __init__(self, dataloc):
        fileNames = [f for f in listdir(dataloc) if isfile(join(dataloc, f)) and f[-4:] == '.mat' and f[:8].isnumeric()]
        fileNames.sort(key=lambda f: int(re.sub('\D', '', f)))
        fileNames = fileNames[:-1] # Cut the last file off as it is an unpredicatble length.

        self.d = [loadmat(dataloc+'\\'+ fileName) for fileName in fileNames]

class data_processing:
    def __init__(self, d, CH):
        self.d = d
    
        self.CH = CH

        self.Ti = d[0]['Tinterval'][0,0]
        self.l = d[0]['Length'][0,0]
        
        self.split_data = []
        self.processed_data = []

    def plot_time_series(self, ax):
        ax.plot(np.linspace(0,self.l*self.Ti,self.l),self.d[0][self.CH],color='tab:red')
        ax.set_title('Time Series Data (CH {})'.format(self.CH))
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Voltage (V)')
        
    def plot_full_PSD(self, ax):
        #plt.figure(figsize=(10, 3.5), dpi=80)
        xf, pxx = periodogram(self.d[0][self.CH][:,0],fs=1/self.Ti)
        ax.semilogy()
        ax.plot(xf,pxx,color='black')
        ax.set_title('PSD (CH {})'.format(self.CH))
        ax.set_xlabel('Freq. (Hz)')
        ax.set_ylabel('Voltage ($V^2/Hz$)')
        
    def splitting_data(self, freq = 3e5, osc = 1e4):
        self.split_data = []
        split_length = int(self.l*self.Ti/(osc*(1/freq)))
        
        for i in range(len(self.d)):
            r = (self.d[0][self.CH])[:,0].shape[0]%split_length
            self.split_data.append(np.split((self.d[0][self.CH])[:-r,0], split_length))
            
        self.split_data = np.asarray(self.split_data)
    
    def average(self):
        # Find Summed PSDs.
        lpxx = []
        # For each channel, adds all periodograms generated from d seperate runs together.
        for i in range(len(self.d)):
            for j in range(self.split_data.shape[1]):
                # This is done to limit too much memory allocation.
                if i == 0: 
                    xf, pxx = periodogram(self.split_data[i,j,:],fs=1/self.Ti)
                else:
                    _, pxx = periodogram(self.split_data[i,j,:],fs=1/self.Ti)
                lpxx.append(pxx[1:])
                
        lpxx = np.asarray(lpxx)
        mpxx = np.mean(lpxx, axis= 0 )
        spxx = np.std(lpxx, axis= 0 )
        epxx = np.std(lpxx, axis= 0 )/np.sqrt(lpxx.shape[0])
        self.processed_data = [xf[1:], mpxx, spxx, epxx]

    def plot_mean_errorbars(self, ax):
        xf, mpxx, epxx = self.processed_data[0], self.processed_data[1], self.processed_data[3]
        ax.semilogy()
        ax.errorbar(xf, mpxx, yerr = epxx, marker='', linestyle='')
        ax.set_title('PSD Data (log10)')
        ax.set_xlabel('Freq. (Hz)')
        ax.set_ylabel('PSD ($V^2/Hz$)')

class fit_data:
    def __init__(self, processed_data):
        self.processed_data = processed_data
        self.peak_params = [[],[]]
        self.popt = []
        self.pcov = []
        self.labels = []
        
        
    def find_peaks(self, width_tuning, ax, peak_lim = 50):
        #width_tuning = 11
        #peak_lim = 50
        peak_list = []
        amp_list = []
        
        xf = self.processed_data[0]
        mpxx = self.processed_data[1]

        #progress_bar(i,len(CH),40)
        #plt.figure(figsize=(10, 3.5), dpi=80)
        ax.semilogy()
        ax.plot(xf, mpxx,color='black')
        ax.set_title('PSD Data (log10)')
        ax.set_xlabel('Freq. (Hz)')
        ax.set_ylabel('PSD ($V^2/Hz$)')
        peaks = find_peaks(mpxx, width = width_tuning)[0]
        print('Peaks: {}'.format(len(peaks)))
        if len(peaks) > peak_lim:
            print('Error: Too Many Peaks')
        for j in range(len(peaks)):
            ax.axvline(x=xf[peaks[j]],color='tab:red')
            peak_list.append(xf[peaks[j]])
            amp_list.append(mpxx[peaks[j]])
        #peaks = np.append(peaks, [1200]) #Temporary additional peak
        #peak_list.append(55000) #Temporary additional peak
        #amp_list.append(2e-9) #Temporary additional peak
        p = len(peak_list)
        self.peak_params = [p, peaks, peak_list, amp_list]
        plt.show()
        
    def fit_multipeak(self, a, b, c, d, gamma, amp, peak_search_area = 250):
        xf, mpxx, spxx, epxx = self.processed_data[0], self.processed_data[1], self.processed_data[2], self.processed_data[3]
        p, peaks, peak_list, amp_list = self.peak_params

        # Build multi func.
        vs = 'omega, a, b, c, d,'
        funcString1 = "def multi_peak({}):"
        funcString2 = ' a/((omega-d)**b) + c +'
        for i in range(p):
            vs+=' omegaq{}, Gammacm{}, a{},'.format(i,i,i)
            funcString2 += ' ssqq(omega, omegaq{}, Gammacm{}, a{}) +'.format(i,i,i)
        funcString2 = funcString2[:-2]
        funcString1 = funcString1.format(vs)
        funcString = funcString1 + '\n return' +  funcString2

        exec(funcString, globals())
        
        lowerBound = ()
        upperBound = ()
        lowerBound += (0,)
        upperBound += (np.inf,)
        lowerBound += (0.01,)
        upperBound += (3,)
        lowerBound += (0,)
        upperBound += (np.inf,)
        lowerBound += (0,)
        upperBound += (np.inf,)
        #Initial Guesses
        p0g = [a, b, c, d]
        for i in range(p):
            lb = peak_list[i]-peak_search_area
            if lb < 0:
                lb = 0
            ub = peak_list[i]+peak_search_area
            if ub > xf[-1]:
                ub = xf[-1]
            lowerBound += (lb,)
            upperBound += (ub,)
            p0g.append(peak_list[i]) # Peak pos guesses.
            lowerBound += (0,)
            upperBound += (np.inf,) 
            p0g.append(gamma) # Gamma Guess
            lowerBound += (0,)
            upperBound += (np.inf,)
            p0g.append(amp_list[i]*amp*1e14) # amp guesses
        #Fitting
        self.popt, self.pcov = curve_fit(multi_peak, xf, mpxx,
                               sigma = spxx, absolute_sigma = True, bounds = (lowerBound, upperBound), p0 = p0g)

    def peak_identification(self, bounds = 1000):
        popt = self.popt
        pcov = self.pcov
        peaks = self.peak_params[1]
        amp_list = self.peak_params[3]
        #Harmonic identification
        #bounds = 1000 #bound on identification for harmonics
        h_order = 1 #max order to identify, fundamental is order 1
        harmonic = np.zeros((len(peaks),h_order))
        for i in range(len(peaks)):
            for j in range(len(peaks)):
                for k in range(h_order):
                    if popt[4+3*i] >= (k+2)*popt[4+3*j] - bounds and popt[4+3*i] <= (k+2)*popt[4+3*j] + bounds:
                        harmonic[i,k] = j
        #Sideband identification
        bounds = 1500 #bound on identification for sidebands
        harm = np.sum(harmonic,1) # used to ignore harmonics
        s_order = 2 #order of sidebands
        s = [y for y in range(-1*s_order, s_order+1, 1) if y!=0]
        sideband = np.zeros((len(peaks),2,2*s_order))
        for i in range(len(peaks)):
            for j in range(len(peaks)):
                for k in range(len(peaks)):
                    for l in range(len(s)):
                        if popt[4+3*i] >= popt[4+3*j] + s[l]*popt[4+3*k] - bounds and popt[4+3*i] <= popt[4+3*j] + s[l]*popt[4+3*k] + bounds and (i!=j and i>k) and j!=k and harm[i] == 0 and harm[j] == 0 and harm[k] == 0 and not (i in harmonic):
                            if amp_list[i] < amp_list[j]: #or amp_list[i] > amp_list[int(sideband[i,1,j])]:
                                sideband[i,0,l] = j
                                sideband[i,1,l] = k
        #removing fake sidebands/finding real peaks
        side = np.sum(sideband, 2)
        for i in range(len(peaks)):
            for j in range(len(s)):
                if sideband[i,0,j] != 0:
                    if side[int(sideband[i,0,j]),0] != 0 or side[int(sideband[i,1,j]),0] != 0:
                        sideband[i,0,j] = 0
                        sideband[i,1,j] = 0
        
        #labelling
        alph = list(string.ascii_uppercase)
        labels = list(alph[0:len(peaks)])
        j = 0
        for i in range(len(peaks)):
            if harm[i] == 0 and side[i,0] == 0:
                labels[i] = alph[j]
                j += 1
        for i in range(len(peaks)):
            if harm[i] != 0:
                for k in range(h_order):
                    labels[i] = str(int((k+2))) + labels[int(harmonic[i,k])];
            elif side[i,0] != 0:
                for k in range(len(s)):
                    if sideband[i,0,k] != 0:
                        if s[k] < 0:
                            sign = '-' + str(int(abs(s[k])))
                        elif s[k] > 0:
                            sign = '+' + str(int(abs(s[k])))
                        if labels[i] != alph[i]:
                            labels[i] += '/' + '\n' + labels[int(sideband[i,0,k])] + sign + labels[int(sideband[i,1,k])]
                        else:
                            labels[i] = labels[int(sideband[i,0,k])] + sign + labels[int(sideband[i,1,k])]
        self.labels = labels
    
    def plot(self, ax, ax2, auto_label = True, label_toggle = True):
        xf, mpxx, spxx, epxx = self.processed_data[0], self.processed_data[1], self.processed_data[2], self.processed_data[3]
        p, peaks, peak_list, amp_list = self.peak_params
        popt = self.popt
        pcov = self.pcov
        if label_toggle and auto_label:
            self.peak_identification()
        elif label_toggle:
            alph = list(string.ascii_uppercase)
            self.labels = list(alph[0:p])
        #Plotting
        points = 350 #Number of points on final graph
        point_sep = round(len(xf)/points)
        #Plot initialisation ax is main psd and fit, ax2 is residual plot
        fig = plt.figure(figsize=(10, 3.5), dpi=150)
        #ax = fig.add_axes((0, 0, 1, 1))
        #ax2 = fig.add_axes((0, -0.21, 1, 0.2))
        ax.semilogy() #y log
        ax.tick_params(axis = 'x', labelsize = 0, bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both')
        ax.tick_params(axis = 'y', bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both')
        ax2.tick_params(bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both')
        ax.set_xlim([0,xf.max()])
        ax2.set_xlim(ax.get_xlim())
        ax2.set_ylim([-50,50])
        ax.set_title('PSD Data')
        ax.set_xlabel('Freq. (Hz)')
        ax.set_ylabel('PSD ($V^2/Hz$)')
        ax.errorbar(xf[::point_sep], mpxx[::point_sep], yerr = epxx[::point_sep], marker='.', linestyle='',
                     color='black', alpha=0.7, ms = 2, linewidth = 0, elinewidth = 1,
                     capsize = 1, ecolor = 'black', zorder = 0) #plot psd
        #for pe in popt[4:][::3]:
        #    ax.axvline(x=pe, color='tab:red', linestyle='dotted') #plot peak markers
        ax.plot(xf, multi_peak(xf,*popt), linewidth = 2, color='tab:red', zorder = 1) #plot multifit
        ax.plot(xf, (popt[0]/((xf-popt[3])**popt[1])) + popt[2], linestyle = '--') #plot background
        for i in range(p):
            #ppos = peaks[i]
            #xax = xf[ppos-peak_search_area:ppos+peak_search_area]
            #ax.plot(xax, ssqq(xax, *popt[4+i*3:4+(i+1)*3]) +  popt[0]/(xax**popt[1]) + popt[2], linestyle = '--')
            ax.fill_between(xf, ssqq(xf, *popt[4+i*3:4+(i+1)*3]) +  popt[0]/(xf**popt[1]) + popt[2], (popt[0]/((xf-popt[3])**popt[1])) + popt[2], alpha = 0.5)
            if label_toggle:
                ax.annotate(self.labels[i],(popt[4+3*i], amp_list[i]), ha='center')
        res = ((mpxx-multi_peak(xf,*popt))/epxx) #calculating residuals
        ax2.scatter(xf[::point_sep], res[::point_sep], color = 'k', s = 0.1) #plotting residuals
        #print(np.sum(res)**2/res.shape[0])
        plt.show()
        #print(80*' ', end = '\r')
        #print('Done.')
        #print(np.sqrt(np.diag(pcov)))
        #print(popt)
        
    def fit_n_plot(self, a, b, c, d, gamma, amp, peak_search_area = 250):
        self.find_peaks(11)
        self.fit_multipeak(a, b, c, d, gamma, amp, peak_search_area = 250)
        self.peak_identification()
        self.plot()
        
    #def save(self,directory,fname):
        

In [9]:
import tkinter as tk
import tkinter.font as tkFont


class App:
    def __init__(self, root):
        # Data processing variables.
        self.loc = ''
        self.d = []
        self.process = None
        self.CH = ''
        
        
        #setting title
        root.title("multifit")
        root.wm_iconbitmap('multifit.ico')
        #setting window size
        width=1200
        height=600
        screenwidth = root.winfo_screenwidth()
        screenheight = root.winfo_screenheight()
        alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth - width) / 2, (screenheight - height) / 2)
        root.geometry(alignstr)
        root.resizable(width=False, height=False)
        root.configure(bg='#333333')
        
        # Title.
        GLabel_268=tk.Label(root)
        ft = tkFont.Font(family='Times',size=20)
        GLabel_268["font"] = ft
        GLabel_268["bg"] = "#333333"
        GLabel_268["fg"] = "#FFFFFF"
        GLabel_268["justify"] = "center"
        GLabel_268["text"] = "multifit"
        GLabel_268.place(x=8,y=6,width=150,height=20)
        
        # Put logo in top corner.
        global new_image
        pic = (Image.open(r"multifit.png"))
        resized_image = pic.resize((30,30), Image.ANTIALIAS)
        new_image= ImageTk.PhotoImage(resized_image)
        label=tk.Label(image=new_image)
        label.pack()
        label.place(x=3,y=3,width=30,height=30)
        
        # Line.
        GMessage_851=tk.Message(root)
        ft = tkFont.Font(family='Times',size=10)
        GMessage_851["borderwidth"] = "0px"
        GMessage_851["font"] = ft
        GMessage_851["fg"] = "#FFFFFF"
        GMessage_851["justify"] = "center"
        GMessage_851["text"] = ""
        GMessage_851.place(x=0,y=36,width=width,height=1)
        
        # File location label.
        self.GLabel_261=tk.Label(root)
        ft = tkFont.Font(family='Times',size=10)
        self.GLabel_261["font"] = ft
        self.GLabel_261["bg"] = "#333333"
        self.GLabel_261["fg"] = "#FFFFFF"
        self.GLabel_261["justify"] = "left"
        self.GLabel_261["anchor"] = "w"
        self.GLabel_261["text"] = "File Path: ..."
        self.GLabel_261.place(x=10,y=47,width=375,height=20)
        
        # File location selector.
        GButton_488=tk.Button(root)
        GButton_488["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_488["font"] = ft
        GButton_488["fg"] = "#FFFFFF"
        GButton_488["justify"] = "center"
        GButton_488["text"] = "f"
        GButton_488.place(x=375,y=43,width=25,height=25)
        GButton_488["command"] = self.GButton_488_command
        
        # Load button.
        GButton_489=tk.Button(root)
        GButton_489["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_489["font"] = ft
        GButton_489["fg"] = "#FFFFFF"
        GButton_489["justify"] = "center"
        GButton_489["text"] = "Load"
        GButton_489.place(x=10,y=73,width=50,height=25)
        GButton_489["command"] = self.GButton_489_command
        
        # Load bar.
        self.GLabel_961=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_961["font"] = ft
        self.GLabel_961["bg"] = "#333333"
        self.GLabel_961["fg"] = "#FFFFFF"
        self.GLabel_961["justify"] = "left"
        self.GLabel_961["anchor"] = "w"
        w = 40
        pr = 0
        self.GLabel_961["text"] = ''
        self.GLabel_961.place(x=60,y=77,width=375,height=20)
        
        # Line.
        GMessage_858=tk.Message(root)
        ft = tkFont.Font(family='Times',size=10)
        GMessage_858["borderwidth"] = "0px"
        GMessage_858["font"] = ft
        GMessage_858["fg"] = "#FFFFFF"
        GMessage_858["justify"] = "center"
        GMessage_858["text"] = ""
        GMessage_858.place(x=0,y=106,width=400,height=1)
        
        # Channel select.
        OPTIONS = [
        "A",
        "B",
        "C",
        "D"    
        ]
        variable = tk.StringVar(root)
        variable.set(OPTIONS[0])
        GOptionMenu_750=tk.OptionMenu(root, variable, *OPTIONS, command=self.callback_750)
        GOptionMenu_750["borderwidth"] = "0px"
        ft = tkFont.Font(family='Times',size=10)
        GOptionMenu_750["font"] = ft
        GOptionMenu_750["bg"] = "#282828"
        GOptionMenu_750["fg"] = "#FFFFFF"
        GOptionMenu_750["justify"] = "center"
        GOptionMenu_750.place(x=10,y=115,width=45,height=25)
        
        # Plot time series button.
        GButton_439=tk.Button(root)
        GButton_439["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_439["font"] = ft
        GButton_439["fg"] = "#FFFFFF"
        GButton_439["justify"] = "center"
        GButton_439["text"] = "Time"
        GButton_439.place(x=60,y=115,width=50,height=25)
        GButton_439["command"] = self.GButton_439_command
        
        # Plot PSD button.
        GButton_139=tk.Button(root)
        GButton_139["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_139["font"] = ft
        GButton_139["fg"] = "#FFFFFF"
        GButton_139["justify"] = "center"
        GButton_139["text"] = "PSD"
        GButton_139.place(x=114,y=115,width=50,height=25)
        GButton_139["command"] = self.GButton_139_command
        
        # Embed matplotlib.
          
        # the figure that will contain the plot
        self.fig = Figure(figsize = (5, 5),
                     dpi = 100)
        #self.fig.set_facecolor((51/255, 51/255, 51/255))

        # adding the subplot.
        #self.plot1 = self.fig.add_subplot(111).
                                       #x0,y0,x1,y1
        self.plot1 = self.fig.add_axes((0.1, 0.3, 1, 1))
        self.plot2 = self.fig.add_axes((0.1, 0.1, 1, 0.3))
        
        self.plot1.tick_params(axis = 'x', labelsize = 0, bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both', colors = 'black')
        self.plot1.tick_params(axis = 'y', bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both', colors = 'black')
        self.plot2.tick_params(bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both', colors = 'black')
        
        self.plot1.set_facecolor((1, 1, 1))
        self.plot2.set_facecolor((1, 1, 1))

        # creating the Tkinter canvas
        # containing the Matplotlib figure
        self.canvas = FigureCanvasTkAgg(self.fig,
                                   master = root)  
        self.canvas.draw()

        # placing the canvas on the Tkinter window
        self.canvas.get_tk_widget().place(x=415,y=45,width=760,height=350)

        # creating the Matplotlib toolbar
        toolbar = NavigationToolbar2Tk(self.canvas,
                                       root)
        toolbar.update()

        # placing the toolbar on the Tkinter window
        self.canvas.get_tk_widget().place(x=420,y=45,width=760,height=350)
        
        # Line.
        GMessage_858=tk.Message(root)
        ft = tkFont.Font(family='Times',size=10)
        GMessage_858["borderwidth"] = "0px"
        GMessage_858["font"] = ft
        GMessage_858["fg"] = "#FFFFFF"
        GMessage_858["justify"] = "center"
        GMessage_858["text"] = ""
        GMessage_858.place(x=168,y=115,width=1,height=55)
        
        # Freq.
        self.GLabel_F=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_F["font"] = ft
        self.GLabel_F["bg"] = "#333333"
        self.GLabel_F["fg"] = "#FFFFFF"
        self.GLabel_F["justify"] = "left"
        self.GLabel_F["anchor"] = "w"
        self.GLabel_F["text"] = 'Max Freq.'
        self.GLabel_F.place(x=173,y=115,width=62,height=20)
        
        # Split Hz input.
        self.GLineEdit_Hz = tk.Entry(root)
        self.GLineEdit_Hz["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_Hz["font"] = ft
        self.GLineEdit_Hz["fg"] = "#FFFFFF"
        self.GLineEdit_Hz["bg"] = "#282828"
        self.GLineEdit_Hz["justify"] = "center"
        self.GLineEdit_Hz["text"] = "Entry0"
        self.GLineEdit_Hz.place(x=238,y=115,width=50,height=23)
        
        # Hz.
        self.GLabel_Hz=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_Hz["font"] = ft
        self.GLabel_Hz["bg"] = "#333333"
        self.GLabel_Hz["fg"] = "#FFFFFF"
        self.GLabel_Hz["justify"] = "left"
        self.GLabel_Hz["anchor"] = "w"
        self.GLabel_Hz["text"] = 'kHz'
        self.GLabel_Hz.place(x=291,y=115,width=25,height=20)
        
        # Osc.
        self.GLabel_Osc=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_Osc["font"] = ft
        self.GLabel_Osc["bg"] = "#333333"
        self.GLabel_Osc["fg"] = "#FFFFFF"
        self.GLabel_Osc["justify"] = "left"
        self.GLabel_Osc["anchor"] = "w"
        self.GLabel_Osc["text"] = 'Osc.'
        self.GLabel_Osc.place(x=173,y=140,width=62,height=20)
        
        # Split oscillation number input.
        self.GLineEdit_Osc = tk.Entry(root)
        self.GLineEdit_Osc["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_Osc["font"] = ft
        self.GLineEdit_Osc["fg"] = "#FFFFFF"
        self.GLineEdit_Osc["bg"] = "#282828"
        self.GLineEdit_Osc["justify"] = "center"
        self.GLineEdit_Osc["text"] = "Entry1"
        self.GLineEdit_Osc.place(x=238,y=140,width=50,height=23)
        
        # Split data button.
        GButton_split_avg=tk.Button(root)
        GButton_split_avg["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_split_avg["font"] = ft
        GButton_split_avg["fg"] = "#FFFFFF"
        GButton_split_avg["justify"] = "center"
        GButton_split_avg["text"] = "Split"
        GButton_split_avg.place(x=293,y=140,width=50,height=25)
        GButton_split_avg["command"] = self.GButton_split_avg_command
        
        # Find peaks button
        GButton_fp_avg=tk.Button(root)
        GButton_fp_avg["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_fp_avg["font"] = ft
        GButton_fp_avg["fg"] = "#FFFFFF"
        GButton_fp_avg["justify"] = "center"
        GButton_fp_avg["text"] = "Peaks"
        GButton_fp_avg.place(x=143,y=210,width=50,height=25)
        GButton_fp_avg["command"] = self.GButton_fp_command
        
        # Width input.
        self.GLineEdit_width = tk.Entry(root)
        self.GLineEdit_width["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_width["font"] = ft
        self.GLineEdit_width["fg"] = "#FFFFFF"
        self.GLineEdit_width["bg"] = "#282828"
        self.GLineEdit_width["justify"] = "center"
        self.GLineEdit_width["text"] = "Entry2"
        self.GLineEdit_width.place(x=88,y=210,width=50,height=23)
        
        # Width.
        self.GLabel_Osc=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_Osc["font"] = ft
        self.GLabel_Osc["bg"] = "#333333"
        self.GLabel_Osc["fg"] = "#FFFFFF"
        self.GLabel_Osc["justify"] = "left"
        self.GLabel_Osc["anchor"] = "w"
        self.GLabel_Osc["text"] = 'Width Tune.'
        self.GLabel_Osc.place(x=10,y=210,width=72,height=20)
        
        # Line.
        GMessage_858=tk.Message(root)
        ft = tkFont.Font(family='Times',size=10)
        GMessage_858["borderwidth"] = "0px"
        GMessage_858["font"] = ft
        GMessage_858["fg"] = "#FFFFFF"
        GMessage_858["justify"] = "center"
        GMessage_858["text"] = ""
        GMessage_858.place(x=0,y=242,width=400,height=1)
        
        # _______________________________________________ #
        of = 8
        # a input.
        self.GLineEdit_a = tk.Entry(root)
        self.GLineEdit_a["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_a["font"] = ft
        self.GLineEdit_a["fg"] = "#FFFFFF"
        self.GLineEdit_a["bg"] = "#282828"
        self.GLineEdit_a["justify"] = "center"
        self.GLineEdit_a["text"] = "Entry3"
        self.GLineEdit_a.place(x=25+of,y=275,width=50,height=23)
        # a.
        self.GLabel_a=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_a["font"] = ft
        self.GLabel_a["bg"] = "#333333"
        self.GLabel_a["fg"] = "#FFFFFF"
        self.GLabel_a["justify"] = "left"
        self.GLabel_a["anchor"] = "center"
        self.GLabel_a["text"] = 'a.'
        self.GLabel_a.place(x=25+of,y=250,width=50,height=20)
        
        # b input.
        self.GLineEdit_b = tk.Entry(root)
        self.GLineEdit_b["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_b["font"] = ft
        self.GLineEdit_b["fg"] = "#FFFFFF"
        self.GLineEdit_b["bg"] = "#282828"
        self.GLineEdit_b["justify"] = "center"
        self.GLineEdit_b["text"] = "Entry4"
        self.GLineEdit_b.place(x=80+of,y=275,width=50,height=23)
        # b.
        self.GLabel_b=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_b["font"] = ft
        self.GLabel_b["bg"] = "#333333"
        self.GLabel_b["fg"] = "#FFFFFF"
        self.GLabel_b["justify"] = "left"
        self.GLabel_b["anchor"] = "center"
        self.GLabel_b["text"] = 'b.'
        self.GLabel_b.place(x=80+of,y=250,width=50,height=20)
        
        # c input.
        self.GLineEdit_c = tk.Entry(root)
        self.GLineEdit_c["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_c["font"] = ft
        self.GLineEdit_c["fg"] = "#FFFFFF"
        self.GLineEdit_c["bg"] = "#282828"
        self.GLineEdit_c["justify"] = "center"
        self.GLineEdit_c["text"] = "Entry5"
        self.GLineEdit_c.place(x=135+of,y=275,width=50,height=23)
        # c.
        self.GLabel_c=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_c["font"] = ft
        self.GLabel_c["bg"] = "#333333"
        self.GLabel_c["fg"] = "#FFFFFF"
        self.GLabel_c["justify"] = "left"
        self.GLabel_c["anchor"] = "center"
        self.GLabel_c["text"] = 'c.'
        self.GLabel_c.place(x=135+of,y=250,width=50,height=20)
        
        # d input.
        self.GLineEdit_d = tk.Entry(root)
        self.GLineEdit_d["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_d["font"] = ft
        self.GLineEdit_d["fg"] = "#FFFFFF"
        self.GLineEdit_d["bg"] = "#282828"
        self.GLineEdit_d["justify"] = "center"
        self.GLineEdit_d["text"] = "Entry6"
        self.GLineEdit_d.place(x=190+of,y=275,width=50,height=23)
        # d.
        self.GLabel_d=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_d["font"] = ft
        self.GLabel_d["bg"] = "#333333"
        self.GLabel_d["fg"] = "#FFFFFF"
        self.GLabel_d["justify"] = "left"
        self.GLabel_d["anchor"] = "center"
        self.GLabel_d["text"] = 'd.'
        self.GLabel_d.place(x=190+of,y=250,width=50,height=20)
        
        # gamma input.
        self.GLineEdit_gamma = tk.Entry(root)
        self.GLineEdit_gamma["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_gamma["font"] = ft
        self.GLineEdit_gamma["fg"] = "#FFFFFF"
        self.GLineEdit_gamma["bg"] = "#282828"
        self.GLineEdit_gamma["justify"] = "center"
        self.GLineEdit_gamma["text"] = "Entry7"
        self.GLineEdit_gamma.place(x=245+of,y=275,width=50,height=23)
        # gammas.
        self.GLabel_gammas=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_gammas["font"] = ft
        self.GLabel_gammas["bg"] = "#333333"
        self.GLabel_gammas["fg"] = "#FFFFFF"
        self.GLabel_gammas["justify"] = "left"
        self.GLabel_gammas["anchor"] = "center"
        self.GLabel_gammas["text"] = 'gamma.'
        self.GLabel_gammas.place(x=245+of,y=250,width=50,height=20)
        
        # amp input.
        self.GLineEdit_amp = tk.Entry(root)
        self.GLineEdit_amp["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.GLineEdit_amp["font"] = ft
        self.GLineEdit_amp["fg"] = "#FFFFFF"
        self.GLineEdit_amp["bg"] = "#282828"
        self.GLineEdit_amp["justify"] = "center"
        self.GLineEdit_amp["text"] = "Entry8"
        self.GLineEdit_amp.place(x=300+of,y=275,width=50,height=23)
        # amp.
        self.GLabel_amps=tk.Label(root)
        ft = tkFont.Font(family='TkFixedFont',size=10)
        self.GLabel_amps["font"] = ft
        self.GLabel_amps["bg"] = "#333333"
        self.GLabel_amps["fg"] = "#FFFFFF"
        self.GLabel_amps["justify"] = "left"
        self.GLabel_amps["anchor"] = "center"
        self.GLabel_amps["text"] = 'amp.'
        self.GLabel_amps.place(x=300+of,y=250,width=50,height=20)
        
        # multifit button.
        GButton_multifit_avg=tk.Button(root)
        GButton_multifit_avg["bg"] = "#282828"
        ft = tkFont.Font(family='Times',size=10)
        GButton_multifit_avg["font"] = ft
        GButton_multifit_avg["fg"] = "#FFFFFF"
        GButton_multifit_avg["justify"] = "center"
        GButton_multifit_avg["text"] = "Fit"
        GButton_multifit_avg.place(x=300+of,y=305,width=50,height=25)
        GButton_multifit_avg["command"] = self.GButton_multifit_command
        

    def GButton_488_command(self):
        #This is to do with finding the correct location of the folder.
        root = tk.Tk() # pointing root to Tk() to use it as Tk() in program.
        root.withdraw() # Hides small tkinter window.
        root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
        path = filedialog.askdirectory() # Returns opened path as str
        short_path = path.split('/')
        lp = len(path)
        if lp > 45:
            out = short_path[0] + '/...' + path[-45:]
        else:
            out = path
        self.GLabel_261["text"] = "File Path: {}".format(out)
        self.loc = path
    
    def GButton_489_command(self):
        dataloc = self.loc
        fileNames = [f for f in listdir(dataloc) if isfile(join(dataloc, f)) and f[-4:] == '.mat' and f[:8].isnumeric()]
        fileNames.sort(key=lambda f: int(re.sub('\D', '', f)))
        fileNames = fileNames[:-1] # Cut the last file off as it is an unpredicatble length.
        loaded = []
        i = 0
        pr = 0
        #L = len(fileNames)
        L = len(fileNames[:2])
        w = 38
        #for fileName in fileNames:
        for fileName in fileNames[:2]:
            self.GLabel_961["text"] = '[{}{}] {}/100%'.format(int(w*pr)*'#',int(w*(1-pr))*'_',round(100*pr,1))
            root.update_idletasks()
            loaded.append(loadmat(dataloc+'\\'+ fileName))
            i+=1
            pr = i/L
        self.GLabel_961["text"] = '[{}{}] {}/100%'.format(int(w*pr)*'#',int(w*(1-pr))*'_',round(100*pr,1))
        self.d = loaded
    
    # Change data set depending on selection box.
    def callback_750(self, selection):
        self.CH = str(selection)
        self.process = data_processing(self.d, self.CH)
    
    # Plot time series.
    def GButton_439_command(self):
        self.plot1.cla()
        self.process.plot_time_series(self.plot1)
        self.canvas.draw()
    
    # Plot PSD
    def GButton_139_command(self):
        self.plot1.cla()
        self.process.plot_full_PSD(self.plot1)
        self.canvas.draw()
        
    def GButton_split_avg_command(self):
        hz = self.GLineEdit_Hz.get()
        osc = self.GLineEdit_Osc.get()
        if str.isdigit(hz) and str.isdigit(osc):
            self.process.splitting_data(freq = int(hz)*1000, osc = int(osc))
        self.processed_data = self.process.average()
        self.plot1.cla()
        self.plot2.cla()
        self.process.plot_mean_errorbars(self.plot1)
        self.canvas.draw()
        
    def GButton_fp_command(self):
        width = int(self.GLineEdit_width.get())
        self.fit = fit_data(self.process.processed_data)
        self.plot1.cla()
        self.plot2.cla()
        self.fit.find_peaks(width, self.plot1)
        self.canvas.draw()
        
    def GButton_multifit_command(self):
        a = int(self.GLineEdit_a.get())
        b = int(self.GLineEdit_b.get())
        c = int(self.GLineEdit_c.get())
        d = int(self.GLineEdit_d.get())
        gamma = 10**int(self.GLineEdit_gamma.get())
        amp = 10**int(self.GLineEdit_amp.get())
        self.plot1.cla()
        self.plot2.cla()
        self.fit.fit_multipeak(a, b, c, d, gamma, amp)
        self.fit.plot(self.plot1, self.plot2)
        self.canvas.draw()
        
        
    def _quit():
        root.quit()     # stops mainloop
        root.destroy()  # this is necessary on Windows to prevent
                        # Fatal Python Error: PyEval_RestoreThread: NULL tstate


In [ ]:
root = tk.Tk()
app = App(root)
root.mainloop()

C:\Users\k2366736\AppData\Local\Temp\ipykernel_18220\3291654555.py:40: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_image = pic.resize((30,30), Image.ANTIALIAS)


In [364]:
gs = Rod_Gamas(300,880e-9,220e-9/2,2.3290*1000,1)

mass_estimate = gs.M
print('Mass estimate:{} kg'.format(mass_estimate))
temp_estimate = gs.T
print('Temperature Estimate:{} K'.format(temp_estimate))

Mass estimate:7.790895940216278e-17 kg
Temperature Estimate:300 K


In [5]:
loc = r'\\samba.nms.kcl.ac.uk\store\millen\OptoMech\880nm Nanorods Polarisation Feedback\20230711TempRun60mW\9mbar'
data = data_loader(loc)
d = data.d

In [8]:
CH = 'D'
process = data_processing(d, CH)
#process.plot_time_series()
#process.plot_full_PSD()
process.splitting_data()
process.average()
processed_data = process.processed_data
#process.plot_mean_errorbars(processed_data)
print('Done.')
# Shopping (slay?)
# pressure loop
# Harmonics

Done.


In [12]:
        fig = Figure(figsize = (5, 5),
                     dpi = 100)
        #self.fig.set_facecolor((51/255, 51/255, 51/255))

        # adding the subplot.
        #self.plot1 = self.fig.add_subplot(111).
                                       #x0,y0,x1,y1
        plot1 = fig.add_axes((0.1, 0.3, 1, 1))
        plot2 = fig.add_axes((0.1, 0.1, 1, 0.3))
        
        plot1.tick_params(axis = 'x', labelsize = 0, bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both', colors = 'black')
        plot1.tick_params(axis = 'y', bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both', colors = 'black')
        plot2.tick_params(bottom = True, top = True, left = True, right = 'True', direction = 'in', which = 'both', colors = 'black')
        
        plot1.set_facecolor((1, 1, 1))
        plot2.set_facecolor((1, 1, 1))

In [ ]:
fit = fit_data(processed_data)
fit.find_peaks(11, plot1)
fit.fit_multipeak(0, 2, 0, 1e0, 1e4, 1)
fit.peak_identification()
fit.plot(plot1, plot2)
#fit.find_peaks(11)
#fit.fit_n_plot(0, 2, 0, 1e0, 1e4, 1)
#fit.fit_multipeak(0, 2, 0, 1e0, 1e4, 1)
plt.show()